<a href="https://colab.research.google.com/github/mahikajain20/LHL_LLM_Project/blob/main/notebooks/4-optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install evaluate
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Th

In [ ]:
!pip install evaluate
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 6.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import transformers as tr
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import hamming_loss, f1_score
from transformers import BertTokenizer, BertModel
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import requests
import getpass
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import evaluate
import accelerate
import tqdm
from accelerate import Accelerator

In [ ]:
import huggingface_hub
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
#loading the dataset
ds = load_dataset('imdb')

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

## TOKENIZER

For this analysis, instead of using manual tokenization techniques I will be using an Autotokenizer and then map it to the whole dataset.

In [ ]:
#Selecting the model and tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Preprocess the dataset
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128) #Changed max length from 128 to 256

In [ ]:
# Tokenize the dataset
tokenized_dataset = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

## MODEL



In this analysis, I will be using DistillBERT as it is faster than the BERT model.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Prepare metric
metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## TRAINING

In [ ]:
# # Setting up training arguments
# training_args = TrainingArguments(
#     output_dir='my_model',
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=1,
#     fp16=True,
#     push_to_hub=False,
#     evaluation_strategy="epoch",
# )

In [ ]:
#Optimize the traiing arguments

training_args = TrainingArguments(
    output_dir='my_model',
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    fp16=True,
    warmup_ratio=0.01, #added
    weight_decay=0.01, #added
    push_to_hub=False,
    evaluation_strategy="epoch",
    optim="adamw_torch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

In [ ]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.053800,0.702022,0.869000


TrainOutput(global_step=1563, training_loss=0.07666290744481297, metrics={'train_runtime': 178.441, 'train_samples_per_second': 140.102, 'train_steps_per_second': 8.759, 'total_flos': 827921241600000.0, 'train_loss': 0.07666290744481297, 'epoch': 1.0})

## EVALUATE


In [ ]:
trainer.evaluate()

{'eval_loss': 0.7020220160484314,
 'eval_accuracy': 0.869,
 'eval_runtime': 32.5199,
 'eval_samples_per_second': 768.76,
 'eval_steps_per_second': 48.063,
 'epoch': 1.0}

In [ ]:
from google.colab import userdata
hf_token = userdata.get('hf_token')

In [ ]:
# Save the model
trainer.save_model("model_1_sentiment")

In [ ]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

events.out.tfevents.1720414996.51c9ca5d910e.1051.0:   0%|          | 0.00/5.97k [00:00<?, ?B/s]

events.out.tfevents.1720415241.51c9ca5d910e.1051.1:   0%|          | 0.00/411 [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Mahikajain/my_model/commit/c86c2a545f9fe0a50d7672cd57d15f3fe7757e96', commit_message='End of training', commit_description='', oid='c86c2a545f9fe0a50d7672cd57d15f3fe7757e96', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the path in Google Drive
drive_model_path = ('/content/drive/MyDrive/Colab Notebooks/model_1_sentiment')

# Copy the model directory to Google Drive
!cp -r ./saved_model $drive_model_path

print(f"Model and tokenizer saved to Google Drive at {drive_model_path}")


Mounted at /content/drive
cp: target 'Notebooks/model_1_sentiment' is not a directory
Model and tokenizer saved to Google Drive at /content/drive/MyDrive/Colab Notebooks/model_1_sentiment


In [ ]:
classifier = pipeline("sentiment-analysis", model="model_1_sentiment")
result = classifier("I love this movie!")
print(result)

[{'label': 'LABEL_1', 'score': 0.9843310713768005}]
